In [1]:
import gdspy
import numpy as np
from element import *
from sketch import *

### Обозначения слоев
0 - металл с копланарами

1 - подложка

2 - металл с чипами

3 - vias

4 - порты

5 - технический уровень

6 - сверление на неполную глубину

# 16-pin держатель

In [2]:
# sketch = Sketch('16-pin sample holder')
# Rcpw = 45e3/2

# R_center_position_hole = 52e3/2
# D_positionHole = 1.6e3

# a_screw = 9e3*2
# D_screw = 4.3e3

# portePerEdge = 4
# portsNum = portePerEdge*4
# singlePortAngle = 2*np.pi/portsNum

# u_cpw = np.array([np.cos(singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2),
#                  -np.sin(singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2)])*Rcpw
# h_position_hole = np.array([np.cos(singlePortAngle/2 + singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2),
#                  -np.sin(singlePortAngle/2 + singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2)])*R_center_position_hole
# holeSize = 11e3

# hole = ChipHole(a = holeSize, portsPerEdge = portePerEdge, portDisp = 1280*2)

# for i in range(0, portsNum):
#     if i%portePerEdge == 0:
#         chipPort = PCB_CHIP(leftVia = True, rightVia = True)
#     else:
#         chipPort = PCB_CHIP(leftVia = True, rightVia = False)
#     chipPort.mergeWithPort(hole.ports[i], 1)
    
#     smpPort = SMP_PCB()
#     smpPort.move(u_cpw)
#     smpPort.rotate(np.pi - (singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2))
#     smpPort.rotate(singlePortAngle*i, center = (0,0))
    
#     if i%4 == 0 or i%4 == 3:
#         cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 350, chessOrder = True)
#     else:
#         cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 550, chessOrder = True)
    
#     positionHole = ThroughHole(D_positionHole)
#     positionHole.move(h_position_hole)
#     positionHole.rotate(singlePortAngle*i, center = (0,0))
    
#     sketch.addGeometry(positionHole)
#     sketch.addGeometry(cpw)
#     sketch.addGeometry(smpPort)
#     sketch.addGeometry(chipPort)

# #     print(i)
    
# for i in [-1,1]:
#     for j in [-1,1]:
#         screwHole = ThroughHole(D_screw)
#         screwHole.move((i*a_screw/2, j*a_screw/2))
#         sketch.addGeometry(screwHole)
        
    
    
# sketch.addGeometry(hole)
# sketch.draw()
# sketch.saveGDS('16pin')
# sketch.saveSVG('16pin')

# 8-pin держатель

In [3]:
sketch = Sketch('8-pin sample holder')
Rcpw = 30e3/2

R_center_position_hole = 30e3/2
D_positionHole = 1.6e3

a_screw = 5.5e3*2
D_screw = 3e3

portePerEdge = 2
portsNum = portePerEdge*4
singlePortAngle = 2*np.pi/portsNum

u_cpw = np.array([np.cos(singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2),
                 -np.sin(singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2)])*Rcpw
h_position_hole = np.array([np.cos(singlePortAngle/2 + singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2),
                 -np.sin(singlePortAngle/2 + singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2)])*R_center_position_hole
holeSize = 6e3

hole = ChipHole(a = holeSize, portsPerEdge = portePerEdge, portDisp = 1280*2)

for i in range(0, portsNum):
    if i%portePerEdge == 0:
        chipPort = PCB_CHIP(leftVia = True, rightVia = True)
    else:
        chipPort = PCB_CHIP(leftVia = True, rightVia = False)
    chipPort.mergeWithPort(hole.ports[i], 1)
    
    smpPort = SMP_PCB()
    smpPort.move(u_cpw)
    smpPort.rotate(np.pi - (singlePortAngle*(portePerEdge-1)//2 + singlePortAngle/2*(portePerEdge-1)%2))
    smpPort.rotate(singlePortAngle*i, center = (0,0))
    

    cpw = CPW(port1 = chipPort.ports[0], port2 = smpPort.ports[0], s = 1000, w = 330,  inter_dist = 400, chessOrder = True)
    
    positionHole = ThroughHole(D_positionHole)
    positionHole.move(h_position_hole)
    positionHole.rotate(singlePortAngle*i, center = (0,0))
    
    sketch.addGeometry(positionHole)
    sketch.addGeometry(cpw)
    sketch.addGeometry(smpPort)
    sketch.addGeometry(chipPort)

#     print(i)
    
for i in [-1,1]:
    for j in [-1,1]:
        screwHole = ThroughHole(D_screw)
        screwHole.move((i*a_screw/2, j*a_screw/2))
        sketch.addGeometry(screwHole)
        
    
    
sketch.addGeometry(hole)
sketch.draw()
sketch.saveGDS('8pin')
sketch.saveSVG('8pin')

C:\Users\burun\Desktop\GDSPY Drawer\sketch.py:84: UserWarning: [GDSPY] Polygons with more than 8190 are not supported by the official GDSII specification.  This GDSII file might not be compatible with all readers.
  self.lib.write_gds(filename + '.gds')
